In [1]:
%load_ext autoreload 
%autoreload 2

In [9]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings

warnings.filterwarnings("ignore")
_ = load_dotenv(find_dotenv())

## Create the fundamental analyst tool

In [2]:
import sys
sys.path.append("../../src")
from fundamental_analyst import FundamentalAnalyst

In [3]:
from typing import Annotated, List, Optional, Literal, Dict
from langchain_core.tools import tool
import pandas as pd

In [4]:
@tool
def evaluate_fundamentals(
    ticker: Annotated[str, "Stock ticker of interest"]
) -> pd.DataFrame:
    """
    This tool is used for fundamental analysis of stock prices.
    """
    fa = FundamentalAnalyst(ticker=ticker)
    df = fa.analyse()
    df = df.fillna(0) 
    return df

## Create the technical analyst tool

In [7]:
from technical_analyst import TechnicalAnalyst

@tool
def evaluate_technicals(
    ticker: Annotated[str, "The stock ticker of interest"],
    period: Annotated[
        Optional[
            Literal[
                "1d",
                "5d",
                "1mo",
                "3mo",
                "6mo",
                "1y",
                "2y",
                "5y",
                "10y",
                "ytd",
                "max"
            ]
        ],
        "The lookback period of interest"
    ] = "10y"
) -> pd.DataFrame:
    """
    This tool is used for technical analysis of stock prices.
    """
    analyst = TechnicalAnalyst()
    return analyst.analyse(ticker=ticker, period=period)

## Create the research tool

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
tavily_tool = TavilySearchResults(max_results=5)

@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )